In [17]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import joblib

In [28]:
dataset = 'Asset15k.xlsx'
df = pd.read_excel(dataset)

In [29]:
X = df.iloc[:, 1:-2]
y = df['AssetType']

In [30]:
X

,SerialNo,Manufacturer,ModelNbr
0,NaN,NaN,NaN
1,TBA,NaN,TBA
2,TBA,NaN,TBA
3,NaN,NaN,NaN
4,TBA,NaN,TBA
...,...,...,...
14470,TBA,COOLAIR,ASH 2000 FMSP
14471,TBA,NaN,TBA
14472,TBA,COOLAIR,ASH 1500 FMSP
14473,TBA,NaN,TBA


In [31]:
# Impute missing values
imputer = SimpleImputer(strategy='most_frequent')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [32]:
X

,SerialNo,Manufacturer,ModelNbr
0,TBA,FANTECH,TBA
1,TBA,FANTECH,TBA
2,TBA,FANTECH,TBA
3,TBA,FANTECH,TBA
4,TBA,FANTECH,TBA
...,...,...,...
14470,TBA,COOLAIR,ASH 2000 FMSP
14471,TBA,FANTECH,TBA
14472,TBA,COOLAIR,ASH 1500 FMSP
14473,TBA,FANTECH,TBA


In [33]:
special_values = [1234, 'UNKOWN', 'Unknown', 'NA', 'NULL', 'UNKNOWN', 'TBA', 'N/A', 'NOT VISIBLE', '123TEST', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na', 'AS PER PICS','nan']

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
X_copy = X.copy()

# Replace values using .loc to avoid SettingWithCopyWarning
X_copy.loc[:, 'SerialNo'] = X_copy['SerialNo'].replace(special_values, pd.NA)
X_copy.loc[:, 'Manufacturer'] = X_copy['Manufacturer'].replace(special_values, pd.NA)
X_copy.loc[:, 'ModelNbr'] = X_copy['ModelNbr'].replace(special_values, pd.NA)

X = X_copy

In [34]:
LE_SerialNo = joblib.load('model/LE_Serial')
LE_ModelNbr = joblib.load('model/LE_ModelNbr')
LE_Manufact = joblib.load('model/LE_Manufact')
LE_Asset = joblib.load('model/LE_Asset')

In [35]:
X['ModelNbr'] = LE_ModelNbr.transform(X['ModelNbr'].astype(str))
X['SerialNo'] = LE_SerialNo.transform(X['SerialNo'].astype(str))
X['Manufacturer'] = LE_Manufact.transform(X['Manufacturer'].astype(str))

y = LE_Asset.transform(y.astype(str))

In [36]:
rf_classifier = joblib.load('model/rf_final2')

In [37]:
y_pred_encoded = rf_classifier.predict(X)

In [38]:
accuracy = accuracy_score(y, y_pred_encoded)
print(f"Accuracy on the chunk: {accuracy}")

Accuracy on the chunk: 0.609119170984456
